<a href="https://colab.research.google.com/github/aa2machiste/aaz/blob/main/Copia_de_Copia_de_colab_models_as_a_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyngrok import ngrok

# El token debe ir entre comillas para ser válido
auth_token = "38C7qOpsgGgphXiJk02QkZLk6rD_3wcqGWSeRpqzjgdKHbDRo"

# Configurar el token
ngrok.set_auth_token(auth_token)
print("\n¡Authtoken configurado! Ahora vuelve a ejecutar la celda de la aplicación Flask para iniciar el servidor.")


¡Authtoken configurado! Ahora vuelve a ejecutar la celda de la aplicación Flask para iniciar el servidor.


# Colab AI: List available models

In [ ]:
from google.colab import ai
ai.list_models()

['google/gemini-2.0-flash',
 'google/gemini-2.0-flash-lite',
 'google/gemini-2.5-flash',
 'google/gemini-2.5-flash-lite',
 'google/gemini-2.5-pro',
 'google/gemini-3-pro-preview',
 'google/gemma-3-12b',
 'google/gemma-3-1b',
 'google/gemma-3-27b',
 'google/gemma-3-4b']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The model names give you a hint about their capabilities and intended use:

Pro: These are the most capable models, ideal for complex reasoning, creative tasks, and detailed analysis.

Flash: These models are optimized for high speed and efficiency, making them great for summarization, chat applications, and tasks requiring rapid responses.

Gemma: These are lightweight, open-weight models suitable for a variety of text generation tasks and are great for experimentation.

# Colab AI: Choose a different model

In [ ]:
from google.colab import ai

response = ai.generate_text("What is the capital of England", model_name='google/gemini-2.0-flash-lite')
print(response)

The capital of England is **London**.



# Colab AI: Simple batch generation example

In [ ]:
# Only text-to-text input/output is supported
from google.colab import ai

response = ai.generate_text("What is the capital of France?")
print(response)

The capital of France is **Paris**.


# Colab AI: Simple streaming example

In [ ]:
from google.colab import ai

stream = ai.generate_text("Tell me a short story.", stream=True)
for text in stream:
  print(text, end='')

Elara bought the house because of its silence. Not an empty silence, but one thick with generations of unspoken stories, like dust motes dancing in sunbeams. It was old, drafty, and smelled faintly of lavender and forgotten paper. Her first task was the attic – a chaotic symphony of trunks, yellowed linens, and furniture draped like sleeping giants.

One sweltering afternoon, she pulled back a heavy canvas sheet from what she assumed was a discarded chest. Instead, her fingers met smooth, dark wood. Beneath the dust and cobwebs lay a mandolin case, surprisingly intact. It wasn't ornate, but beautifully crafted, its dark wood worn soft in places, hinting at countless hours in someone's hands.

With a click, the latches sprang open. Inside, nestled on faded velvet, was the mandolin. Its strings were loose, spiderwebbed with dust, and one tuning peg was missing. But even in disarray, it held a quiet grace. A faint, sweet scent, like dried rose petals, clung to it.

Elara carefully lifted 

# Colab AI: Formatted streaming example

In [ ]:
#code is not necessary for colab.ai, but is useful in fomatting text chunks
import sys
from google.colab import ai


class LineWrapper:
    def __init__(self, max_length=80):
        self.max_length = max_length
        self.current_line_length = 0

    def print(self, text_chunk):
        i = 0
        n = len(text_chunk)
        while i < n:
            start_index = i
            while i < n and text_chunk[i] not in ' \n': # Find end of word
                i += 1
            current_word = text_chunk[start_index:i]

            delimiter = ""
            if i < n: # If not end of chunk, we found a delimiter
                delimiter = text_chunk[i]
                i += 1 # Consume delimiter

            if current_word:
                needs_leading_space = (self.current_line_length > 0)

                # Case 1: Word itself is too long for a line (must be broken)
                if len(current_word) > self.max_length:
                    if needs_leading_space: # Newline if current line has content
                        sys.stdout.write('\n')
                        self.current_line_length = 0
                    for char_val in current_word: # Break the long word
                        if self.current_line_length >= self.max_length:
                            sys.stdout.write('\n')
                            self.current_line_length = 0
                        sys.stdout.write(char_val)
                        self.current_line_length += 1
                # Case 2: Word doesn't fit on current line (print on new line)
                elif self.current_line_length + (1 if needs_leading_space else 0) + len(current_word) > self.max_length:
                    sys.stdout.write('\n')
                    sys.stdout.write(current_word)
                    self.current_line_length = len(current_word)
                # Case 3: Word fits on current line
                else:
                    if needs_leading_space:
                        # Define punctuation that should not have a leading space
                        # when they form an entire "word" (token) following another word.
                        no_leading_space_punctuation = {
                            ",", ".", ";", ":", "!", "?",        # Standard sentence punctuation
                            ")", "]", "}",                     # Closing brackets
                            "'s", "'S", "'re", "'RE", "'ve", "'VE", # Common contractions
                            "'m", "'M", "'ll", "'LL", "'d", "'D",
                            "n't", "N'T",
                            "...", "…"                          # Ellipses
                        }
                        if current_word not in no_leading_space_punctuation:
                            sys.stdout.write(' ')
                            self.current_line_length += 1
                    sys.stdout.write(current_word)
                    self.current_line_length += len(current_word)

            if delimiter == '\n':
                sys.stdout.write('\n')
                self.current_line_length = 0
            elif delimiter == ' ':
                # If line is full and a space delimiter arrives, it implies a wrap.
                if self.current_line_length >= self.max_length:
                    sys.stdout.write('\n')
                    self.current_line_length = 0

        sys.stdout.flush()


wrapper = LineWrapper()
for chunk in ai.generate_text('Give me a long winded description about the evolution of the Roman Empire.', model_name='google/gemini-2.0-flash', stream=True):
  wrapper.print(chunk)

Alright, buckle up, because charting the evolution of the Roman Empire is like
navigating a sprawling, millennia-old labyrinth. It's a story of humble
beginnings, audacious expansion, catastrophic crises, and ultimately, a slow,
multi-faceted disintegration. Forget easy chapters; this is more like a tapestry
woven with ambition, innovation, brutality, and an enduring legacy that
continues to shape our world.

We begin not with emperors and legions, but with the *Romulus and Remus* myth, a
foundation story as much symbolic as historical. Around the 8th century BCE, a
collection of small villages nestled on the Palatine Hill, overlooking the Tiber
River, coalesced into what would eventually become Rome. This was the era of the
Roman Kingdom, a period shrouded in legend but likely characterized by a
succession of kings, possibly influenced by the Etruscans to the north. While we
know less about this era than later periods, it was crucial in establishing
foundational societal structures, r

# Task
Generate text for multiple prompts using the `colab.ai` API to demonstrate sequential text generation capabilities, and clarify the current limitations of `colab.ai` regarding external API exposure.

## Clarify colab.ai Capabilities

### Subtask:
Explain the current capabilities of the `colab.ai` API, focusing on its use for text generation within the Colab environment and clarifying that direct external API exposure via tools like ngrok is not natively supported.


The `colab.ai` API is specifically designed to facilitate text generation tasks directly within the Google Colab environment. It provides a convenient way to access and utilize various large language models (LLMs) for tasks like content creation, summarization, and interactive AI assistance, all contained within your Colab notebook.

**Important Note:** Currently, `colab.ai` does not support direct external API exposure. This means that you cannot use `colab.ai` to host models for external applications or services via tools like ngrok or similar tunneling services. Its functionality is strictly confined to the Colab runtime where it is invoked.

The `colab.ai` API is specifically designed to facilitate text generation tasks directly within the Google Colab environment. It provides a convenient way to access and utilize various large language models (LLMs) for tasks like content creation, summarization, and interactive AI assistance, all contained within your Colab notebook.

**Important Note:** Currently, `colab.ai` does not support direct external API exposure. This means that you cannot use `colab.ai` to host models for external applications or services via tools like ngrok or similar tunneling services. Its functionality is strictly confined to the Colab runtime where it is invoked.

The `colab.ai` API is specifically designed to facilitate text generation tasks directly within the Google Colab environment. It provides a convenient way to access and utilize various large language models (LLMs) for tasks like content creation, summarization, and interactive AI assistance, all contained within your Colab notebook.

**Important Note:** Currently, `colab.ai` does not support direct external API exposure. This means that you cannot use `colab.ai` to host models for external applications or services via tools like ngrok or similar tunneling services. Its functionality is strictly confined to the Colab runtime where it is invoked.

## Prepare Multiple Prompts

### Subtask:
Define a list of several prompts to simulate 'batching' requests for text generation from a model.


**Reasoning**:
I need to define a list of prompts for text generation as per the instructions. This will involve creating a Python list named `prompts` and populating it with distinct text prompts.



In [ ]:
prompts = [
    "What is the capital of Canada?",
    "Write a short poem about a cat.",
    "Explain the concept of machine learning in simple terms.",
    "What are the benefits of eating healthy?"
]

print(f"Created a list of {len(prompts)} prompts:")
for i, prompt in enumerate(prompts):
    print(f"{i+1}. {prompt}")

Created a list of 4 prompts:
1. What is the capital of Canada?
2. Write a short poem about a cat.
3. Explain the concept of machine learning in simple terms.
4. What are the benefits of eating healthy?


**Reasoning**:
The subtask of defining multiple prompts has been completed. The next step is to use these prompts to generate text sequentially, as indicated by the main task "demonstrate sequential text generation capabilities."



In [ ]:
from google.colab import ai

print("--- Generating responses for multiple prompts ---")
responses = []
for i, prompt in enumerate(prompts):
    print(f"\nPrompt {i+1}: {prompt}")
    response = ai.generate_text(prompt)
    responses.append(response)
    print(f"Response {i+1}: {response}")

print("\n--- All responses generated ---")

--- Generating responses for multiple prompts ---

Prompt 1: What is the capital of Canada?
Response 1: The capital of Canada is **Ottawa**.

Prompt 2: Write a short poem about a cat.
Response 2: A shadow soft, a silent grace,
Pads through life at its own pace.
A sleepy blink, a gentle purr,
Beneath a coat of silken fur.
Then sudden dash, a playful leap,
Secrets that the night will keep.
A furry friend, both wild and mild,
A cherished, curious, whiskered child.

Prompt 3: Explain the concept of machine learning in simple terms.
Response 3: Imagine you want a computer to do something, but instead of telling it *exactly* how to do it step-by-step (like a normal computer program), you let it **learn how to do it itself** from examples, much like a human or a child learns.

Here's the breakdown in simple terms:

1.  **Learning from Experience:**
    Think about teaching a child to recognize a cat. You don't give them a long list of rules like "it has fur, four legs, whiskers, meows, a tail

## Final Task

### Subtask:
Summarize the demonstration of generating text for multiple prompts and reiterate the current limitations regarding external API exposure for `colab.ai` models.


## Summary:

### Q&A
1.  **Summary of generating text for multiple prompts:** The process successfully demonstrated sequential text generation by defining a list of four distinct prompts (e.g., "What is the capital of Canada?", "Write a short poem about a cat.") and then iterating through them to generate a response for each using `colab.ai.generate_text()`. The generated responses were coherent and relevant to their respective prompts, showcasing the API's ability to handle diverse text generation requests.
2.  **Current limitations regarding external API exposure for `colab.ai` models:** `colab.ai` does not currently support direct external API exposure. This means it cannot be used to host models for external applications or services via tunneling services like ngrok, as its functionality is strictly confined to the Google Colab runtime environment.

### Data Analysis Key Findings
*   The `colab.ai` API is designed exclusively for text generation tasks within the Google Colab environment, providing access to large language models for various content creation and AI assistance needs.
*   **A significant limitation is that `colab.ai` does not support direct external API exposure, meaning it cannot be used with external services or tools like ngrok for hosting models.**
*   The `colab.ai` API effectively generated coherent and relevant text responses for multiple diverse prompts sequentially, including factual questions, creative writing, and explanatory requests.

### Insights or Next Steps
*   Given the current limitations, `colab.ai` is best suited for in-notebook prototyping, experimentation, and educational purposes rather than production-level external application integration.
*   Future enhancements could include exploring secure, controlled methods for limited external API access or publishing generated content, expanding the utility of `colab.ai` beyond the Colab runtime.


# Task
Instalar las bibliotecas Flask y pyngrok y crear una API Flask sencilla con al menos un endpoint que utilice `colab.ai` internamente para generar texto, para luego exponer esta API públicamente usando ngrok, probando el acceso externo y resumiendo las limitaciones de `colab.ai` respecto a la exposición directa de API.

## Instalar dependencias

### Subtask:
Instalar las bibliotecas Flask y pyngrok necesarias en el entorno de Colab.


**Reasoning**:
To install the required libraries Flask and pyngrok, I need to execute `pip install` commands in a code block.



In [ ]:
!pip install Flask
!pip install pyngrok

## Crear API Flask Sencilla

### Subtask:
Escribir el código Python para definir una aplicación Flask básica con al menos un endpoint. Este endpoint puede, por ejemplo, devolver un mensaje simple o procesar una entrada.


**Reasoning**:
I need to write Python code to define a basic Flask application with a root endpoint that returns a simple string message, as per the instructions.



In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return "¡Hola desde Flask en Colab!"

# To run Flask in Colab, you'll typically use ngrok to expose it, but this basic setup
# focuses on defining the application logic first. Actual running with ngrok will be in the next step.
# For now, we just define the app.

**Reasoning**:
The previous code block defined the Flask application but did not include the execution logic. As per the instructions, I need to add the `if __name__ == '__main__':` block to run the Flask app on port 5000, which is a necessary step before exposing it via ngrok.



In [ ]:
from flask import Flask, jsonify
import flask
from google.colab import ai
from pyngrok import ngrok
import os
import time
import subprocess
import signal

app = Flask(__name__)

# Definir el endpoint raíz
@app.route('/')
def home():
    return "¡La API de Colab AI está activa! Usa /generate?prompt=... para probarla."

# Definir el endpoint que usa colab.ai
@app.route('/generate', methods=['GET', 'POST'])
def generate():
    req = flask.request
    prompt = req.args.get('prompt') or (req.json.get('prompt') if req.is_json else None)

    if not prompt:
        return jsonify({'error': 'Por favor proporciona un parametro prompt'}), 400

    try:
        generated_text = ai.generate_text(prompt)
        return jsonify({
            'prompt': prompt,
            'response': generated_text
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    # 1. Configurar Token
    # (Usamos el token que proporcionaste para asegurar que funcione directo)
    auth_token = "38C7qOpsgGgphXiJk02QkZLk6rD_3wcqGWSeRpqzjgdKHbDRo"
    ngrok.set_auth_token(auth_token)

    # 2. LIMPIEZA NUCLEAR DE PROCESOS ANTERIORES
    print("--- Buscando y eliminando procesos ngrok zombies ---")
    ngrok.kill()
    try:
        # Buscar PIDs de ngrok para matarlos definitivamente
        result = subprocess.check_output(["pgrep", "-f", "ngrok"])
        pids = result.decode().split()
        for pid in pids:
            try:
                os.kill(int(pid), signal.SIGKILL)
                print(f"🔫 Proceso ngrok eliminado forzosamente: {pid}")
            except OSError:
                pass
    except subprocess.CalledProcessError:
        print("✅ No se encontraron procesos ngrok activos en el sistema.")

    # Esperar a que el sistema operativo libere los puertos
    print("Esperando unos segundos para liberar puertos...")
    time.sleep(3)

    # 3. Iniciar Túnel
    try:
        public_url = ngrok.connect(5000).public_url
        print(f"\n✅ API PÚBLICA DISPONIBLE EN: {public_url}")
        print(f"   Prueba: {public_url}/generate?prompt=Hola")

        print("\n--- RESUMEN DE LIMITACIONES DE COLAB.AI ---")
        print("1. colab.ai solo funciona dentro del runtime de Colab.")
        print("2. Requiere túneles (ngrok) para acceso externo.")
        print("3. La API muere si se cierra este notebook.")
        print("-------------------------------------------\n")

        app.run(port=5000)

    except Exception as e:
        print(f"\n⚠️ ERROR AL INICIAR: {e}")
        print("Si el error persiste, por favor reinicia el entorno (Runtime -> Restart session).")

--- Buscando y eliminando procesos ngrok zombies ---
✅ No se encontraron procesos ngrok activos en el sistema.
Esperando unos segundos para liberar puertos...



⚠️ ERROR AL INICIAR: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: The endpoint 'https://unomitted-teodora-unennobling.ngrok-free.dev' is already online. Either\n1. stop your existing endpoint first, or\n2. start both endpoints with `--pooling-enabled` to load balance between them.\r\n\r\nERR_NGROK_334\r\n"}}

Si el error persiste, por favor reinicia el entorno (Runtime -> Restart session).
